In [1]:
import pandas as pd
import numpy as np
import re
import sys

# File contains the British pound symbol, an alternate encoding needed to be utilized. 
scholar_file_df = pd.read_csv("/Users/jacquelynzuker/dev/Python/WELLCOME/WELLCOME_APCspend2013_forThinkful.csv", encoding='cp437')
df = pd.DataFrame(scholar_file_df)

In [2]:
# Changed the name of the column
df["COST"] = df["COST (ú) charged to Wellcome (inc VAT when charged)"]
df = df.drop(["COST (ú) charged to Wellcome (inc VAT when charged)"],1)
df.loc[df["COST"].str.contains("\$")>0]

,PMID/PMCID,Publisher,Journal title,Article title,COST
178,PMC3398262,BioMed Central,BMC Biology,Detailed interrogation of trypanosome cell bio...,1674$
179,PMID: 24020915 PMC3846689,BioMed Central,BMC Genetics,The physical capability of community-based men...,1375.8$
180,PMID:23442822,BioMed Central,BMC Genome Biology,Patterns of prokaryotic lateral gene transfers...,2010$
181,PMC2843621,BioMed Central,BMC Genomics,Trichomonas vaginalis vast BspA-like gene fami...,1204.38$
182,PMCID:\n PMC3636053,BioMed Central,BMC Genomics,Enhancing the utility of Proteomics Signature ...,1254.6$
183,3526451,Biomed Central,BMC Genomics,Advances in genome-wide RNAi cellular screens:...,1476$
337,23931322 PMCID: PMC3736666,Byophysical Society,Biophysical Journal,Aggregation modulators interfer with membrane ...,671.04$
1599,23308065,Public Library of Science,PLOS Pathogens,Transmission of equine influenza virus during ...,1440$
1600,PMID:23633946,Public Library of Science,PLoS Pathogens,The Mnn2 Mannosyltransferase Family Modulates ...,1460.3$
1601,PMC3610638,Public Library of Science,PLoS Pathogens,DNA break site at fragile subtelomeres determi...,1476.47$


There are a number of titles where the cost has been inputted in US dollars ($) instead of British Pounds. If any of these titles appear in the top Journals analyzed below, we'll convert the COST into Pounds using the conversion rate of the year this dataset was published. 

In [3]:
# Extracted the numeric value of the cost
df["COST"] = df["COST"].str.extract('(\d+.\d+)').astype(float)
len(df["Journal title"].unique())

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


985

To start, there are 985 unique Journal Titles. There are differences in capitalization, abbreviation, and spelling present in this dataset. Let's see if we can reduce the number of Journals by reducing these differences. 

In [4]:
# Make all of the capitalization identical. 
df["Journal title"] = df["Journal title"].str.upper()
df["Journal title"].value_counts()

PLOS ONE                                                      190
JOURNAL OF BIOLOGICAL CHEMISTRY                                53
NEUROIMAGE                                                     29
PLOS GENETICS                                                  24
PLOS PATHOGENS                                                 24
NUCLEIC ACIDS RESEARCH                                         23
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES                20
PLOS NEGLECTED TROPICAL DISEASES                               20
NATURE COMMUNICATIONS                                          19
HUMAN MOLECULAR GENETICS                                       19
BMC PUBLIC HEALTH                                              14
BRAIN                                                          14
MOVEMENT DISORDERS                                             13
JOURNAL OF NEUROSCIENCE                                        12
BIOCHEMICAL JOURNAL                                            12
DEVELOPMEN

In [5]:
# Based on inspection of the unique journals, the articles are dropped due to variation on whether these have been included.
# Replacements concerning certain abbreviations are included. 
df["Journal title"] = df["Journal title"].str.replace("J ", "JOURNAL ")
df["Journal title"] = df["Journal title"].str.replace("JNL", "JOURNAL ")
df["Journal title"] = df["Journal title"].str.replace("PLOSONE", "PLOS ONE")
df["Journal title"] = df["Journal title"].str.replace(" OF ", " ")
df["Journal title"] = df["Journal title"].str.replace(" THE ", " ")
df["Journal title"] = df["Journal title"].str.replace(" IN ", " ")
df["Journal title"] = df["Journal title"].str.replace(" AND ", " ")
df["Journal title"] = df["Journal title"].str.replace(" & ", " ")

df["Journal title"] = df["Journal title"].str.replace(" FOR ", " ")
df["Journal title"] = df["Journal title"].str.replace("^THE ", " ")
df["Journal title"] = df["Journal title"].str.replace("  ", " ")
df["Journal title"] = df["Journal title"].str.replace('NUCLEIC ACID RESEARCH', 'NUCLEIC ACIDS RESEARCH') 
df["Journal title"] = df["Journal title"].str.replace('UNITED STATES AMERICA', "USA")
df["Journal title"] = df["Journal title"].str.replace("INT ", "INTERNATIONAL ")
df["Journal title"] = df["Journal title"].str.replace("MOL ", "MOLECULAR ")
df["Journal title"] = df["Journal title"].str.replace("HUM ", "HUMAN ")
df["Journal title"] = df["Journal title"].str.replace("BIOL ", "BIOLOGY ")
df["Journal title"] = df["Journal title"].str.replace("BIO ", "BIOLOGY ")
df["Journal title"] = df["Journal title"].str.replace("SCI ", "SCIENCE ")
df["Journal title"] = df["Journal title"].str.replace("PUBLIC LIBRARY SCIENCE", "PLOS")
df["Journal title"] = df["Journal title"].str.replace("BMJ$", "BRITISH MEDICAL JOURNAL ")
df["Journal title"] = df["Journal title"].str.replace("CHEM ", "CHEMISTRY ")
df["Journal title"] = df["Journal title"].str.replace("CHEM$", "CHEMISTRY ")
df["Journal title"] = df["Journal title"].str.replace(" ORG ", " ORGANIC ")

df.ix[df["Journal title"].str.match(".*INT.*AL EPID*")>0, "Journal title"] = "INTERNATIONAL JOURNAL EPIDEMIOLOGY"
df.ix[df["Journal title"].str.match(".*NATIONAL ACADEMY SCIENCES.*")>0, "Journal title"] = "PNAS"
df.ix[df["Journal title"].str.match(".*NAT.* ACA.*")>0, "Journal title"] = "PNAS"
df.ix[df["Journal title"].str.match(".*PNAS.*")>0, "Journal title"] = "PNAS"

df.ix[df["Journal title"].str.match(".*TROPICAL DIS.*")>0, "Journal title"] = "PLOS NEGLECTED TROPICAL DISEASE"
df.ix[df["Journal title"].str.match("HUMAN.*GEN.*")>0, "Journal title"] = "HUMAN MOLECULAR GENETICS"
df.ix[df["Journal title"].str.match("NEUROLMAGE*")>0, "Journal title"] = "NEUROIMAGE"


# Remove spaces ON ENDS
df["Journal title"] = df["Journal title"].str.strip()

df["Journal title"].value_counts()

PLOS ONE                                                        201
JOURNAL BIOLOGICAL CHEMISTRY                                     64
PNAS                                                             40
NEUROIMAGE                                                       31
NUCLEIC ACIDS RESEARCH                                           29
PLOS PATHOGENS                                                   24
PLOS GENETICS                                                    24
PLOS NEGLECTED TROPICAL DISEASE                                  23
HUMAN MOLECULAR GENETICS                                         21
NATURE COMMUNICATIONS                                            19
BMC PUBLIC HEALTH                                                15
JOURNAL NEUROSCIENCE                                             15
BRAIN                                                            14
BIOCHEMICAL JOURNAL                                              14
MOVEMENT DISORDERS                              

In [6]:
len(df["Journal title"].unique())

828

By making a few select changes, we have reduced the number of journal titles by nearly 16%. 

The five most popular journals are PLOS One with 201 articles, Journal of Biological Chemistry with 64 articles, PNAS with 39 articles, Neuroimage with 31 articles, and Nucleic Acids Research with 29 articles. 

In [7]:
df["Journal title"].value_counts()

PLOS ONE                                                        201
JOURNAL BIOLOGICAL CHEMISTRY                                     64
PNAS                                                             40
NEUROIMAGE                                                       31
NUCLEIC ACIDS RESEARCH                                           29
PLOS PATHOGENS                                                   24
PLOS GENETICS                                                    24
PLOS NEGLECTED TROPICAL DISEASE                                  23
HUMAN MOLECULAR GENETICS                                         21
NATURE COMMUNICATIONS                                            19
BMC PUBLIC HEALTH                                                15
JOURNAL NEUROSCIENCE                                             15
BRAIN                                                            14
BIOCHEMICAL JOURNAL                                              14
MOVEMENT DISORDERS                              

Find the statistics including mean, median, and standard deviation of the cost of each of the top five journals below. 

In [8]:
df_stats = df['COST'].groupby(df['Journal title']).describe()

In [9]:
print("\n PLOS ONE: \n\n", df_stats["PLOS ONE"])
print("\n\n JOURNAL BIOLOGICAL CHEMISTRY: \n\n", df_stats["JOURNAL BIOLOGICAL CHEMISTRY"])
print("\n\n PNAS: \n\n", df_stats["PNAS"])
print("\n\n NEUROIMAGE: \n\n", df_stats["NEUROIMAGE"])
print("\n\n NUCLEIC ACIDS RESEARCH: \n\n", df_stats["NUCLEIC ACIDS RESEARCH"])


 PLOS ONE: 

 count       201.000000
mean      41649.768060
std      196068.801781
min         122.310000
25%         850.880000
50%         897.190000
75%        1044.730000
max      999999.000000
Name: COST, dtype: float64


 JOURNAL BIOLOGICAL CHEMISTRY: 

 count        64.000000
mean      32592.024063
std      175125.433955
min         265.670000
25%        1152.190000
50%        1313.130000
75%        1602.602500
max      999999.000000
Name: COST, dtype: float64


 PNAS: 

 count        40.000000
mean      25818.861250
std      157981.566997
min         206.320000
25%         644.935000
50%         742.920000
75%         836.727500
max      999999.000000
Name: COST, dtype: float64


 NEUROIMAGE: 

 count      31.000000
mean     2212.181290
std       273.193244
min      1747.160000
25%      2030.895000
50%      2326.430000
75%      2408.680000
max      2518.230000
Name: COST, dtype: float64


 NUCLEIC ACIDS RESEARCH: 

 count      29.000000
mean     1162.344828
std       442.15093